In [25]:
# Imports
import numpy as np
from sympy import symbols, solve, Poly, simplify, latex
import random
# import sympy as sp

# Main functions

### Function to generate random polynomials
Generates a polynomial of up to a given degree with a specific number of terms. The coefficients of the polynomial are within the specified bounds.

In [1]:
def generate_polynomial(max_degree, num_terms, coeff_bounds):
    """
    Inputs:
    max_degree (int): The maximum degree of the polynomial.
    num_terms (int)
    coeff_bounds (tuple): (lower bound, upper bound)

    Outputs:
    coefficients (list)
    """
    # Initialize array of coeffs
    coefficients = [0] * (max_degree + 1)
    # Check number of terms
    if num_terms > max_degree + 1:
        raise ValueError("Number of terms cannot be more than the degree of the polynomial + 1")
    # Randomly choose positions for the (#) non-zero coefficients
    non_zero_positions = random.sample(range(max_degree), num_terms - 1)
    # Assign random values within the bounds to the chosen positions
    for pos in non_zero_positions:
        # Loop to make sure it is nonzero bc selecting from [-, +] includes 0
        while coefficients[pos] is None or coefficients[pos] == 0:
          coefficients[pos] = random.randint(coeff_bounds[0], coeff_bounds[1])
    # Ensure the highest degree term is non-zero
    if coefficients[-1] == 0:
        # While loop until it is nonzero
        while coefficients[-1] is None or coefficients[-1] == 0:
          coefficients[-1] = random.randint(coeff_bounds[0], coeff_bounds[1])

    return coefficients

In [69]:
# Test function
generate_polynomial(max_degree=10, num_terms=3, coeff_bounds=[-10,10])

[-7, 6, 0, 0, 0, 0, 0, 0, 0, 0, -10]

### Nondimensionalization function
Takes in array of coefficients and returns in a nondimensionalized form (only works for 3-term polynomials due to how it reduces the 2nd and third terms to have coefficients of 1).



In [72]:
def nondimensionalize_polynomial(coefficients):
    """
    Inputs:
    coefficients(list): List of coefficients of a polynomial

    Outputs:
    new_coefficients (list): List of coefficients for equivalent non-dimensionalized polynomial
    """
    # Identify nonzero coefficients and their powers
    nonzero_coeffs = [(coeff, len(coefficients) - idx - 1) for idx, coeff in enumerate(coefficients) if coeff != 0]

    # Ensure there are exactly three nonzero coefficients
    if len(nonzero_coeffs) != 3:
        return "Polynomial must have exactly three nonzero coefficients.", None

    # Find largest, second largest powers
    sorted_coeffs = sorted(nonzero_coeffs, key=lambda x: x[1], reverse=True)
    max_power = sorted_coeffs[0][1]
    second_max_power = sorted_coeffs[1][1]

    # Compute ε
    epsilon = sorted_coeffs[0][0] / sorted_coeffs[1][0]

    # Construct the nondimensionalized polynomial
    signs = [1 if coeff[0] > 0 else -1 for coeff in sorted_coeffs]

    # Printing polynomial (commenting out because not needed)
    # nondimensionalized_poly = f"{epsilon:.2f}x^{max_power} "
    # nondimensionalized_poly += f"{'+' if signs[1] == 1 else '-'} x^{second_max_power} "
    # nondimensionalized_poly += f"{'+' if signs[2] == 1 else '-'} 1"

    # Construct new coeffs array
    new_coefficients = [0] * (max_power + 1)
    new_coefficients[max_power] = epsilon
    new_coefficients[second_max_power] = signs[1]
    new_coefficients[0] = signs[2]
    new_coefficients = list(reversed(new_coefficients))

    # Return
    # return new_coefficients, nondimensionalized_poly
    return new_coefficients

In [71]:
# Example usage
polynomial = [0, 0, -8, 0, 5, 0, 0, 0, 0, 0, 7]
nondimensionalize_polynomial(polynomial)

[-1.6, 0, 1, 0, 0, 0, 0, 0, 1]

In [75]:
print(np.roots([-7, 6, 0, 0, 0, 0, 0, 0, 0, 0, -10]))
print(np.roots([-1.6, 0, 1, 0, 0, 0, 0, 0, 1]))

[ 1.11737589+0.29180969j  1.11737589-0.29180969j  0.70135589+0.80048499j
  0.70135589-0.80048499j  0.07413512+1.00975443j  0.07413512-1.00975443j
 -0.92181274+0.31530581j -0.92181274-0.31530581j -0.54248273+0.82319706j
 -0.54248273-0.82319706j]
[-1.04767631+0.j         -0.71023674+0.59814896j -0.71023674-0.59814896j
  1.04767631+0.j          0.71023674+0.59814896j  0.71023674-0.59814896j
  0.        +0.8751764j   0.        -0.8751764j ]


### Generate nondimensionalized problem
Main solving function for nondimensionalized problem of the form $\epsilon x^{n_1} \pm x^{n_2} \pm 1$ where $n_1 > n_2$

In [2]:
def generate_nondimensionalized(max_n1):
    """
    Inputs:
    max_n1 (int): Maximum power in polynomial

    Outputs:
    polynomial (sympy expression): Nondimensionalized random polynomial of the given format
    """
    # Ensure max_n1 is at least 2
    if max_n1 < 2:
        raise ValueError("max_n1 must be at least 2")

    # Randomly choose n1 and n2
    n1 = random.randint(2, max_n1)
    n2 = random.randint(1, n1 - 1)
    # Randomly choose signs
    signs = random.choices([-1, 1], k=2)

    # Construct polynomial
    x = symbols('x')
    epsilon = symbols('epsilon')
    polynomial = epsilon * x**n1 + signs[0] * x**n2 + signs[1]

    # Return
    return polynomial

In [3]:
# Example usage
polynomial = generate_nondimensionalized(10)

list(polynomial.expand().args)[::-1]

[epsilon*x**9, x**7, 1]

### Solve nondimensionalized problem

We use the following for dominant balances:


In [31]:
# Generate a polynomial
polynomial = generate_nondimensionalized(10)

In [32]:
# Extract terms and solve
x, epsilon = symbols('x epsilon')
terms = list(polynomial.expand().args)[::-1]
A, B, C = terms[0], terms[1], terms[2]
sol_ab = [simplify(sol) for sol in solve(A + B, x)]
sol_bc = [simplify(sol) for sol in solve(B + C, x)]
sol_ac = [simplify(sol) for sol in solve(A + C, x)]

# Remove extraneous root 0 that shows up - bc it never occurs in this problem formulation
# Helper function to remove zeros
def remove_zeros(sol_list):
    while 0 in sol_list:
        sol_list.remove(0)
remove_zeros(sol_ab)
remove_zeros(sol_bc)
remove_zeros(sol_ac)

# Check dominant balances to see if roots belong to small or large epsilon regimes
AB_valid_small_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 0.001)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 0.001))) # A,B >> C small eps
AB_valid_large_eps = (abs(A.subs(x, sol_ab[0]).subs(epsilon, 1000)) > abs(C.subs(x, sol_ab[0]).subs(epsilon, 1000))) # A,B >> C large eps
print(AB_valid_small_eps)
print(AB_valid_large_eps)

BC_valid_small_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 0.001)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 0.001))) # B,C >> A small eps
BC_valid_large_eps = (abs(B.subs(x, sol_bc[0]).subs(epsilon, 1000)) > abs(A.subs(x, sol_bc[0]).subs(epsilon, 1000))) # B,C >> A large eps
print(BC_valid_small_eps)
print(BC_valid_large_eps)

AC_valid_small_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 0.001)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 0.001))) # A,C >> B small eps
AC_valid_large_eps = (abs(A.subs(x, sol_ac[0]).subs(epsilon, 1000)) > abs(B.subs(x, sol_ac[0]).subs(epsilon, 1000))) # A,C >> B large eps
print(AC_valid_small_eps)
print(AC_valid_large_eps)

# TODO: Figure out how to go backwards from multiple complex roots/roots of unity to simpler format (e.g. 1/eps**5)


# Display results
print("polynomial:")
display(polynomial)
print("\n")

print("Balance A, B | ", len(sol_ab), "roots")
display(sol_ab)
print("\n")

print("Balance B, C | ", len(sol_bc), "roots")
display(sol_bc)
print("\n")

print("Balance A, C | ", len(sol_ac), "roots")
display(sol_ac)
print("\n")

True
False
True
False
False
True
polynomial:


epsilon*x**7 + x**5 + 1



Balance A, B |  2 roots


[-sqrt(-1/epsilon), sqrt(-1/epsilon)]



Balance B, C |  5 roots


[-1,
 1/4 + sqrt(5)/4 + sqrt(-10 + 2*sqrt(5))/4,
 -sqrt(5)/4 + 1/4 - I*sqrt(50 - 10*sqrt(5))/8 - I*sqrt(10 - 2*sqrt(5))/8,
 1/4 + sqrt(5)/4 - I*sqrt(10*sqrt(5) + 50)/16 - I*sqrt(2*sqrt(5) + 10)/16 - I*sqrt(10 - 2*sqrt(5))/16 + I*sqrt(50 - 10*sqrt(5))/16,
 -sqrt(5)/4 + 1/4 - sqrt(-10 + 2*sqrt(5))/16 + sqrt(-10 - 2*sqrt(5))/16 + sqrt(-50 + 10*sqrt(5))/16 + sqrt(-50 - 10*sqrt(5))/16]



Balance A, C |  7 roots


[(-1/epsilon)**(1/7),
 -(-1/epsilon)**(1/7)*exp(I*pi/7),
 -(-1/epsilon)**(1/7)*exp(-I*pi/7),
 (-1/epsilon)**(1/7)*exp(-2*I*pi/7),
 (-1/epsilon)**(1/7)*exp(2*I*pi/7),
 -(-1/epsilon)**(1/7)*(sin(pi/14) + I*cos(pi/14)),
 (-1/epsilon)**(1/7)*(-sin(pi/14) + I*cos(pi/14))]

In [ ]:
adef solve_nondimensionalized(polynomial):
    """
    Inputs:
    polynomial (sympy expression): Nondimensionalized polynomial

    Outputs:
    small x sols (sympy expression): Small x analytical solutions in terms of epsilon
    large x sols (sympy expression): Large x analytical solutions in terms of epsilon
    """
    # Separate into separate terms, assign to A,B,C
    terms = list(polynomial.expand().args)[::-1]
    A = terms[0]
    B = terms[1]
    C = terms[2]

    # Try three different dominant balances
    # For each dominant balance:
    # 1) Solve equation for x in terms of epsilon
    # 2) Check that balance in terms
    # 3) Either append to small x solutions or large x solutions.

    # A = B

    # A = C

    # B = C

### Function to generate latex problem statement

In [27]:
\noindent\hrulefill
problem = "Solve for analytical approximaitions to the roots of the following polynomial. Present the roots for both large and small \epsilon."
latex(polynomial)

'\\epsilon x^{9} - x^{4} + 1'

In [29]:
# Define the file name
file_name = "problem_dataset.tex"

# Write to file
with open(file_name, "w") as file:
    file.write("\\documentclass{article}\n")
    file.write("\\usepackage{amsmath}\n")
    file.write("\\begin{document}\n")
    file.write("\\section*{Quadratic Equation}\n")
    file.write("The quadratic equation is given by:\n")
    file.write("\\[" + latex(polynomial) + "\\]\n")
    file.write("\\end{document}")

### Helper function to generate sympy polynomial

In [31]:
# Generates sympy polynomial from
def polynomial_from_coefficients(coefficients):
    """
    Converts a list of coefficients into a polynomial expression using sympy.

    Parameters:
    coefficients (list): A list of coefficients, where the index represents the power of x.

    Returns:
    sympy expression: The polynomial expression.
    """
    x = symbols('x')
    coefficients_reverse = coefficients
    coefficients_reverse.reverse()
    polynomial = sum(coef * x**i for i, coef in enumerate(coefficients_reverse))
    return Poly(polynomial)

In [32]:
# Test functions
coefficients = generate_polynomial(degree=10, num_terms=3, coeff_bounds=[-10,10])
print("Coefficients", coefficients)

polynomial_expression = polynomial_from_coefficients(coefficients)
polynomial_expression

Coefficients [0, 0, -8, 0, 5, 0, 0, 0, 0, 0, 7]


Poly(-8*x**8 + 5*x**6 + 7, x, domain='ZZ')

## Generate latex for problem and solution

## Check that analytical solutions are approx. equal to the numerical solution.

If valid, generate latex -> add to dataset
- Check for duplicates??

Discard problem otherwise

## Unit tests for each function

### Generate polynomial
- Generate several polynomials, check that there are exactly n nonzero coefficients and that there is a constant.

### Nondimensionalize polynomial
- Generate many polynomials, nondimensionalize, compare numerically-solved roots to check they are the same (but scaled)????????????

In [97]:
def test_generate(max_degree, num_terms, coeff_bounds):
  # Test 100 different runs of the function
  for i in range(100):
    # Solve
    coeffs = generate_polynomial(max_degree=max_degree, num_terms=num_terms, coeff_bounds=coeff_bounds)
    # Check length of array
    assert len(coeffs) == max_degree + 1
    # Check number of terms
    assert len([x for x in coeffs if x != 0]) == num_terms
    # Check for nonzero constant term
    assert coeffs[-1] != 0
    # Check that coeff bounds are correct
    all(coeff_bounds[0] <= x <= coeff_bounds[1] for x in coeffs)
  print("All tests passed! (Generate polynomial function)")

In [102]:
# Parameters
max_degree = 10
num_terms = 3
coeff_bounds=[-10,10]

# Run tests
test_generate(max_degree, num_terms, coeff_bounds)

All tests passed! (Generate polynomial function)
